In [73]:
import pandas as pd
import numpy as np
import nltk
import time
import random
import math

In [4]:
df = pd.read_csv('cleaned_tweets.csv',encoding='ISO-8859-1', sep='\,', header=None, usecols=range(8)).apply(lambda x: x.str.replace(r"\"",""))

df.columns = df.iloc[0]
df.drop(index=0, inplace=True)
df
print (df[590:600])
print (df.shape)

df = df.dropna() # still 24000 left of 26000, seems fine to me
print (df.shape)


c:\users\jan_h_000\miniconda3\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


0     ï»¿index                                              count hate_speech  \
591        533                                                  3           0   
592        534                                                  3           0   
593        535                                                  3           2   
594        536                                                  3           1   
595        537                                                  3           0   
596  &#128514;  my nips are freeeeezing and that s not okay be...        None   
597        538                                                  3           0   
598        539                                                  3           0   
599        540                                                  3           2   
600        541                                                  3           0   

0   offensive_language neither class  \
591                  3       0     1   
592                  1      

In [39]:
from sklearn.model_selection import train_test_split

classes = df['class'].values
tweets_full = df['clean_tweet'].values
tweets = []
for tweet in tweets_full:
    tweets.append(tweet.split())
print(tweets[0])

tweets_train, tweets_test, classes_train, classes_test = train_test_split(tweets, classes)
print(classes_train)

train_data = []
for i in range(len(tweets_train)):
    data = (tweets_train[i], classes_train[i])
    train_data.append(data)

test_data = []
for i in range(len(tweets_test)):
    data = (tweets_test[i], classes_test[i])
    test_data.append(data)


['rt', 'as', 'a', 'woman', 'you', 'shouldn', 't', 'complain', 'about', 'cleaning', 'up', 'your', 'house', 'amp', 'as', 'a', 'man', 'you', 'should', 'always', 'take', 'the', 'trash', 'out']
['1' '1' '1' ... '1' '1' '1']


In [19]:


# Here we first define a class that can map a word to an ID (w2i)
# and back (i2w).

from collections import Counter, OrderedDict, defaultdict


class OrderedCounter(Counter, OrderedDict):
  """Counter that remembers the order elements are first seen"""
  def __repr__(self):
    return '%s(%r)' % (self.__class__.__name__,
                      OrderedDict(self))
  def __reduce__(self):
    return self.__class__, (OrderedDict(self),)


class Vocabulary:
  """A vocabulary, assigns IDs to tokens"""
  
  def __init__(self):
    self.freqs = OrderedCounter()
    self.w2i = {}
    self.i2w = []

  def count_token(self, t):
    self.freqs[t] += 1
    
  def add_token(self, t):
    self.w2i[t] = len(self.w2i)
    self.i2w.append(t)    
    
  def build(self, min_freq=0):
    self.add_token("<unk>")  # reserve 0 for <unk> (unknown words)
    self.add_token("<pad>")  # reserve 1 for <pad> (discussed later)   
    
    tok_freq = list(self.freqs.items())
    tok_freq.sort(key=lambda x: x[1], reverse=True)
    for tok, freq in tok_freq:
      if freq >= min_freq:
        self.add_token(tok)
        
# This process should be deterministic and should have the same result 
# if run multiple times on the same data set.

v = Vocabulary()
for tweet in tweets:
    for word in tweet:
        v.count_token(word)

v.build()
print("Vocabulary size:", len(v.w2i))
print(v.i2w)

Vocabulary size: 23916
['<unk>', '<pad>', 'a', 'bitch', 'i', 'the', 'rt', 'you', 'to', 'that', 'and', 'my', 't', '128514', 'bitches', 'in', 'is', 'like', 'me', 'on', 'of', 'it', 'hoes', 'be', 'this', 's', 'pussy', 'for', 'hoe', 'all', 'with', 'ass', '8220', 'up', 'but', 'just', 'they', 'your', 'get', 'fuck', 'm', 'these', 'so', 'u', 'shit', 'when', '8221', 'can', 'got', 'she', 'if', 'don', 'no', 'nigga', 'trash', 'lol', 'her', 'not', 'was', 'are', 'at', 'out', 'he', 'have', 'some', 'about', 'what', 'ain', 'do', 'amp', 'niggas', 'know', 'we', '8230', 'I', 'how', 'them', 'who', 'or', 'love', 'one', 'as', 'fucking', 'yo', 'go', '128557', 'why', 'bad', 'now', 'then', 'his', 'want', 'from', 're', 'good', 'too', 'man', 'ya', 'still', 'look', 'y', 'back', 'only', 'make', 'im', 'off', 'think', 'hate', 'need', 'see', 'say', 'faggot', 'an', 'time', 'will', 'never', 'really', 'real', 'lmao', 'let', 'would', 'right', 'being', 'people', 'there', 'girl', 'even', 'bird', 'than', 'here', 'said', '1285

In [20]:
import torch
from torch import optim
from torch import nn

In [21]:
embed = nn.Embedding(len(v.w2i), 100)
print(embed)

Embedding(23916, 100)


In [22]:
glove_data = []

print("----GLOVE----")
line_number = 0
with open('glove.txt', mode="r", encoding="utf-8") as f:
    for line in f:
        line = line.split()
        glove_data.append(line)
        if (line_number < 4):
            print(line)
            line_number += 1

v_glove_dic = {}
v_glove_dic['<unk>'] = np.random.rand(len(glove_data[0][1:]))
v_glove_dic['<pad>'] = np.zeros(len(glove_data[0][1:]))

for line in glove_data:
    v_glove_dic[line[0]] = line[1:]



v_glove = Vocabulary()
for tweet in tweets:
    for word in tweet:
        if word in v_glove_dic:
            v_glove.count_token(word)
      

v_glove.build()


vectors = []
counter = 0
for line in v_glove.i2w:
    if (len(v_glove_dic[line])) != 300:
        vectors.append(v_glove_dic[line])


vectors = np.stack(vectors, axis=0).astype(np.float)

# copy pre-trained word vectors into embeddings table
# model.embed.weight.data.copy_(torch.from_numpy(vectors))

----GLOVE----
[',', '-0.082752', '0.67204', '-0.14987', '-0.064983', '0.056491', '0.40228', '0.0027747', '-0.3311', '-0.30691', '2.0817', '0.031819', '0.013643', '0.30265', '0.0071297', '-0.5819', '-0.2774', '-0.062254', '1.1451', '-0.24232', '0.1235', '-0.12243', '0.33152', '-0.006162', '-0.30541', '-0.13057', '-0.054601', '0.037083', '-0.070552', '0.5893', '-0.30385', '0.2898', '-0.14653', '-0.27052', '0.37161', '0.32031', '-0.29125', '0.0052483', '-0.13212', '-0.052736', '0.087349', '-0.26668', '-0.16897', '0.015162', '-0.0083746', '-0.14871', '0.23413', '-0.20719', '-0.091386', '0.40075', '-0.17223', '0.18145', '0.37586', '-0.28682', '0.37289', '-0.16185', '0.18008', '0.3032', '-0.13216', '0.18352', '0.095759', '0.094916', '0.008289', '0.11761', '0.34046', '0.03677', '-0.29077', '0.058303', '-0.027814', '0.082941', '0.1862', '-0.031494', '0.27985', '-0.074412', '-0.13762', '-0.21866', '0.18138', '0.040855', '-0.113', '0.24107', '0.3657', '-0.27525', '-0.05684', '0.34872', '0.011884

In [67]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [89]:
# Here we print each parameter name, shape, and if it is trainable.
def print_parameters(model):
  total = 0
  for name, p in model.named_parameters():
    total += np.prod(p.shape)
    print("{:24s} {:12s} requires_grad={}".format(name, str(list(p.shape)), p.requires_grad))
  print("\nTotal parameters: {}\n".format(total))

def prepare_example(example, vocab):
  """
  Map tokens to their IDs for 1 example
  """
  tweet, clas = example
  # vocab returns 0 if the word is not there
  x = [vocab.w2i.get(t, 0) for t in tweet]
  
  x = torch.LongTensor([x])
  x = x.to(device)
  
  y = torch.LongTensor([float(clas)])
  y = y.to(device)
  
  return x, y


def simple_evaluate(model, data, prep_fn=prepare_example, **kwargs):
  """Accuracy of a model on given data set."""
  correct = 0
  total = 0
  model.eval()  # disable dropout (explained later)

  for example in data:
    
    # convert the example input and label to PyTorch tensors
    x, target = prep_fn(example, model.vocab)

    # forward pass
    # get the output from the neural network for input x
    with torch.no_grad():
      logits = model(x)
    
    # get out the prediction
    prediction = logits.argmax(dim=-1)
    
    # add the number of correct predictions to the total correct
    correct += (prediction == target).sum().item()
    total += 1

  return correct, total, correct / float(total)

def get_examples(data, shuffle=True, **kwargs):
  """Shuffle data set and return 1 example at a time (until nothing left)"""
  if shuffle:
    print("Shuffling training data")
    random.shuffle(data)  # shuffle training data each epoch
  for example in data:
    yield example
    
def train_model(model, optimizer, num_iterations=1000, 
                print_every=100, eval_every=100,
                batch_fn=get_examples, 
                prep_fn=prepare_example,
                eval_fn=simple_evaluate,
                batch_size=1, eval_batch_size=None):
  """Train a model."""  
  iter_i = 0
  train_loss = 0.
  print_num = 0
  start = time.time()
  criterion = nn.CrossEntropyLoss() # loss function
  best_eval = 0.
  best_iter = 0
  
  # store train loss and validation accuracy during training
  # so we can plot them afterwards
  losses = []
  accuracies = []  
  
  if eval_batch_size is None:
    eval_batch_size = batch_size
  while True:  # when we run out of examples, shuffle and continue
    for batch in batch_fn(train_data, batch_size=batch_size):
      # forward pass
      model.train()
      x, targets = prep_fn(batch, model.vocab)

      logits = model(x)

      B = targets.size(0)  # later we will use B examples per update
      
      # compute cross-entropy loss (our criterion)
      # note that the cross entropy loss function computes the softmax for us
      loss = criterion(logits.view([B, -1]), targets.view(-1))
      train_loss += loss.item()
      # backward pass
      # Tip: check the Introduction to PyTorch notebook.

      # reset gradients
      model.zero_grad()

      # compute gradients
      loss.backward()

      # update weights - take a small step in the opposite dir of the gradient
      optimizer.step()

      print_num += 1
      iter_i += 1

      # print info
      if iter_i % print_every == 0:
        print("Iter %r: loss=%.4f, time=%.2fs" % 
              (iter_i, train_loss, time.time()-start))
        losses.append(train_loss)
        print_num = 0        
        train_loss = 0.

      # evaluate
      if iter_i % eval_every == 0:
        _, _, accuracy_test = eval_fn(model, test_data, batch_size=eval_batch_size,
                         batch_fn=batch_fn, prep_fn=prep_fn)
        accuracies.append(accuracy_test)
        print("iter %r: test acc=%.4f" % (iter_i, accuracy_test))       
        
        # save best model parameters
        if accuracy_test > best_eval:
          print("new highscore")
          best_eval = accuracy_test
          best_iter = iter_i
          path = "{}.pt".format(model.__class__.__name__)
          ckpt = {
              "state_dict": model.state_dict(),
              "optimizer_state_dict": optimizer.state_dict(),
              "best_eval": best_eval,
              "best_iter": best_iter
          }
          torch.save(ckpt, path)

      # done training
      if iter_i == num_iterations:
        print("Done training")
        
        # evaluate on train, dev, and test with best model
        print("Loading best model")
        path = "{}.pt".format(model.__class__.__name__)        
        ckpt = torch.load(path)
        model.load_state_dict(ckpt["state_dict"])
        
        _, _, train_acc = eval_fn(
            model, train_data, batch_size=eval_batch_size, 
            batch_fn=batch_fn, prep_fn=prep_fn)
        _, _, test_acc = eval_fn(
            model, test_data, batch_size=eval_batch_size, 
            batch_fn=batch_fn, prep_fn=prep_fn)
        
        print("best model iter {:d}: "
              "train_acc={:.4f}, test acc={:.4f}".format(
                  best_iter, train_acc, test_acc))
        
        return losses, accuracies

In [90]:
class MyLSTMCell(nn.Module):
  """Our own LSTM cell"""

  def __init__(self, input_size, hidden_size, bias=True):
    """Creates the weights for this LSTM"""
    super(MyLSTMCell, self).__init__()

    self.input_size = input_size
    self.hidden_size = hidden_size
    self.bias = bias
    
    self.W_i = nn.Linear(input_size, 4*hidden_size, bias = self.bias)
    self.W_h = nn.Linear(hidden_size, 4*hidden_size, bias = self.bias)

    
    
    
    self.reset_parameters()

  def reset_parameters(self):
    """This is PyTorch's default initialization method"""
    stdv = 1.0 / math.sqrt(self.hidden_size)
    for weight in self.parameters():
      weight.data.uniform_(-stdv, stdv)  

  def forward(self, input_, hx, mask=None):
    """
    input is (batch, input_size)
    hx is ((batch, hidden_size), (batch, hidden_size))
    """
    prev_h, prev_c = hx

    # project input and prev state
    x = input_
    # YOUR CODE HERE
    
    total = (self.W_i(x) + self.W_h(prev_h))
    i,f,g,o = torch.chunk(total, 4 , dim=1)
    
    # main LSTM computation    
    i = i.sigmoid()
    f = f.sigmoid()
    g = g.tanh()
    o = o.sigmoid()

    c = f*prev_c + i * g
    h = o * c.tanh()

    return h, c
  
  def __repr__(self):
    return "{}({:d}, {:d})".format(
        self.__class__.__name__, self.input_size, self.hidden_size)

In [91]:
class LSTMClassifier(nn.Module):
  """Encodes sentence with an LSTM and projects final hidden state"""

  def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, vocab):
    super(LSTMClassifier, self).__init__()
    self.vocab = vocab
    self.hidden_dim = hidden_dim
    self.embed = nn.Embedding(vocab_size, embedding_dim, padding_idx=1)
    self.rnn = MyLSTMCell(embedding_dim, hidden_dim)
    
    self.output_layer = nn.Sequential(     
        nn.Dropout(p=0.5),  # explained later
        nn.Linear(hidden_dim, output_dim)
    )

  def forward(self, x):
    
    B = x.size(0)  # batch size (this is 1 for now, i.e. 1 single example)
    T = x.size(1)  # time (the number of words in the sentence)
    
    input_ = self.embed(x)

    # here we create initial hidden states containing zeros
    # we use a trick here so that, if input is on the GPU, then so are hx and cx
    hx = input_.new_zeros(B, self.rnn.hidden_size)
    cx = input_.new_zeros(B, self.rnn.hidden_size)
    
    # process input sentences one word/timestep at a time
    # input is batch-major, so the first word(s) is/are input_[:, 0]
    outputs = []   
    for i in range(T):
      hx, cx = self.rnn(input_[:, i], (hx, cx))
      outputs.append(hx)
    
    # if we have a single example, our final LSTM state is the last hx
    if B == 1:
      final = hx
    else:
      #
      # This part is explained in next section, ignore this else-block for now.
      #
      # we processed sentences with different lengths, so some of the sentences
      # had already finished and we have been adding padding inputs to hx
      # we select the final state based on the length of each sentence
      
      # two lines below not needed if using LSTM form pytorch
      outputs = torch.stack(outputs, dim=0)          # [T, B, D]
      outputs = outputs.transpose(0, 1).contiguous()  # [B, T, D]
      
      # to be super-sure we're not accidentally indexing the wrong state
      # we zero out positions that are invalid
      pad_positions = (x == 1).unsqueeze(-1)
      
      outputs = outputs.contiguous()      
      outputs = outputs.masked_fill_(pad_positions, 0.)
        
      mask = (x != 1)  # true for valid positions [B, T]
      lengths = mask.sum(dim=1)                  # [B, 1]

      indexes = (lengths - 1) + torch.arange(B, device=x.device, dtype=x.dtype) * T
      final = outputs.view(-1, self.hidden_dim)[indexes]  # [B, D]
    
    # we use the last hidden state to classify the sentence
    logits = self.output_layer(final)
    return logits

In [92]:
lstm_model = LSTMClassifier(len(v.w2i), 300, 168, 3, v)

# copy pre-trained word vectors into embeddings table
# with torch.no_grad():
#     lstm_model.embed.weight.data.copy_(torch.from_numpy(vectors))
#     lstm_model.embed.weight.requires_grad = False

print(lstm_model)
print_parameters(lstm_model)

optimizer = optim.Adam(lstm_model.parameters(), lr=3e-4)

lstm_losses, lstm_accuracies = train_model(
    lstm_model, optimizer, num_iterations=2500, 
    print_every=100, eval_every=100)

LSTMClassifier(
  (embed): Embedding(23916, 300, padding_idx=1)
  (rnn): MyLSTMCell(300, 168)
  (output_layer): Sequential(
    (0): Dropout(p=0.5)
    (1): Linear(in_features=168, out_features=3, bias=True)
  )
)
embed.weight             [23916, 300] requires_grad=True
rnn.W_i.weight           [672, 300]   requires_grad=True
rnn.W_i.bias             [672]        requires_grad=True
rnn.W_h.weight           [672, 168]   requires_grad=True
rnn.W_h.bias             [672]        requires_grad=True
output_layer.1.weight    [3, 168]     requires_grad=True
output_layer.1.bias      [3]          requires_grad=True

Total parameters: 7491147

Shuffling training data
Iter 100: loss=100.4133, time=23.88s
iter 100: test acc=0.7344
new highscore
Iter 200: loss=78.7927, time=71.97s
iter 200: test acc=0.7696
new highscore
Iter 300: loss=54.4128, time=117.08s
iter 300: test acc=0.7726
new highscore


KeyboardInterrupt: 